In [30]:
import os
import numpy as np 

from utils.read_data import read_data
from utils.re_referencing import rref_REST, rref_remove, rref_CSD, rref_average
from utils.data_filtering import filter_band_raw_to_raw, filter_band_array_to_array
from utils.data_filtering import eliminate_blink_corr_electrodes
from utils.data_filtering import clean_windows_artifacts, reconstruct_signal

from metrics import analyse_network_by_metric

In [2]:
def create_tt(window_time: int, n_samples: int, i_window: int):
    return np.linspace(0, window_time, n_samples)+(i_window*window_time)

In [5]:
# Params load data
data_folder = "../data/"

# Params Reconstruction
reconst_threshold = 0.2

# Params Cleaning
window_time = 10
clean_threshold = 0.6

# Params AIS
i_window = 0
max_lag = 20

settings = {
     'cmi_estimator':  'OpenCLKraskovCMI',
     'max_lag': max_lag,
     'local_values': True,
     'n_perm_max_stat': 11,
     'n_perm_min_stat': 11,
     'alpha_max_stat': 0.1,
     'n_perm_omnibus': 11,
     'alpha_omnibus': 0.1,
     'n_perm_max_seq': 11,
     'alpha_max_seq': 0.1, 
     'alpha_min_stat': 0.1,
     'verbose': False
}

In [26]:
hc_filenames = list(filter(lambda x: x.startswith("h"), sorted(os.listdir(data_folder))))
sz_filenames = list(filter(lambda x: x.startswith("s"), sorted(os.listdir(data_folder))))

filenames = hc_filenames[:3] + sz_filenames[:3]
filename = filenames[0]
filename

'h01.edf'

In [31]:
# Load data, filter on 0.1-100 [Hz] and rref REST
raw = read_data(os.path.join(data_folder, filename))
raw_filtered = filter_band_raw_to_raw(raw, [0.1, 100])
raw_rref = rref_REST(raw_filtered)

## with raw_filteres (not re-ref)
reconst_raw, ica, eliminated_comps = reconstruct_signal(raw_filtered, reconst_threshold, ch_names=["Fp1", "Fp2"]) #[ 0  1  5  6 13 16]

# Cleaning
data_windows, rejected = clean_windows_artifacts(reconst_raw, 
                                                 window_time=window_time, 
                                                 std_threshold=clean_threshold, 
                                                 rref_fn=rref_REST
                                                )
all_data = np.concatenate(data_windows, axis=1)

for i_channel in [16]: #[16, 17, 18]
    # Calculating AIS
    n_samples = data_windows.shape[-1]
    tt = create_tt(window_time, n_samples, i_window)
    net_analysis = analyse_network_by_metric(data_windows[i_window], channels=i_channel, metric="AIS", **settings)
    ais = net_analysis.get_single_process(0)['ais'][0]

Extracting EDF parameters from /home/camilojd/Universidad/Primavera_2020/EL7006/dynamicinfo-eeg/data/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 8251 samples (33.004 sec)

Fitting ICA to data using 19 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
    Rejecting  epoch based on EEG : ['O1']
Artifact det

INFO:root:window number 9 rejected
INFO:root:window number 28 rejected
INFO:root:window number 32 rejected
INFO:root:window number 33 rejected
INFO:root:window number 36 rejected
INFO:root:window number 58 rejected
INFO:root:window number 61 rejected
INFO:root:window number 64 rejected
INFO:root:window number 67 rejected
INFO:root:window number 70 rejected
INFO:root:window number 73 rejected
INFO:root:window number 81 rejected
INFO:root:window number 82 rejected
INFO:root:window number 83 rejected
INFO:root:window number 84 rejected
DEBUG:pyopencl.cache:build program: binary cache hit (key: af95eb07a92eba2c2ba429500126f90f)
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: disk cache hit [key=9830e4e464ac850c53a15d29fb1ea0822392841646effb7ab7351bc85139fdf0]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: disk cache hit [key=b0be9d43833a6a67bdabcaf743eafe38855ffce6660ec5502b12c36615acb806]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: disk cache hit [key=b4efd

Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data


DEBUG:pyopencl.cache:build program: binary cache hit (key: af95eb07a92eba2c2ba429500126f90f)
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=9830e4e464ac850c53a15d29fb1ea0822392841646effb7ab7351bc85139fdf0]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b0be9d43833a6a67bdabcaf743eafe38855ffce6660ec5502b12c36615acb806]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b4efd189bde0f677b380b90ad80abcdf9858b798940c0e9f9b0bfc8b6beb4077]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=fb92d9fdefe95aeac01927058216ade6f72b3d21cc609bafdbf44c1b956191a5]
DEBUG:pyopencl.cache:build program: binary cache hit (key: af95eb07a92eba2c2ba429500126f90f)
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=9830e4e464ac850c53a15d29fb1ea0822392841646effb7ab7351bc85139fdf0]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b0be9d4383


---------------------------- (1) include candidates



DEBUG:pyopencl.cache:build program: binary cache hit (key: af95eb07a92eba2c2ba429500126f90f)
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=9830e4e464ac850c53a15d29fb1ea0822392841646effb7ab7351bc85139fdf0]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b0be9d43833a6a67bdabcaf743eafe38855ffce6660ec5502b12c36615acb806]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b4efd189bde0f677b380b90ad80abcdf9858b798940c0e9f9b0bfc8b6beb4077]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=fb92d9fdefe95aeac01927058216ade6f72b3d21cc609bafdbf44c1b956191a5]
DEBUG:pyopencl.cache:build program: binary cache hit (key: af95eb07a92eba2c2ba429500126f90f)
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=9830e4e464ac850c53a15d29fb1ea0822392841646effb7ab7351bc85139fdf0]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b0be9d4383

DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b0be9d43833a6a67bdabcaf743eafe38855ffce6660ec5502b12c36615acb806]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b4efd189bde0f677b380b90ad80abcdf9858b798940c0e9f9b0bfc8b6beb4077]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=fb92d9fdefe95aeac01927058216ade6f72b3d21cc609bafdbf44c1b956191a5]
DEBUG:pyopencl.cache:build program: binary cache hit (key: af95eb07a92eba2c2ba429500126f90f)
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=9830e4e464ac850c53a15d29fb1ea0822392841646effb7ab7351bc85139fdf0]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b0be9d43833a6a67bdabcaf743eafe38855ffce6660ec5502b12c36615acb806]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b4efd189bde0f677b380b90ad80abcdf9858b798940c0e9f9b0bfc8b6beb4077]
DEBUG:pytools.persistent_dict:pyopenc


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics


DEBUG:pyopencl.cache:build program: binary cache hit (key: af95eb07a92eba2c2ba429500126f90f)
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=9830e4e464ac850c53a15d29fb1ea0822392841646effb7ab7351bc85139fdf0]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b0be9d43833a6a67bdabcaf743eafe38855ffce6660ec5502b12c36615acb806]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b4efd189bde0f677b380b90ad80abcdf9858b798940c0e9f9b0bfc8b6beb4077]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=fb92d9fdefe95aeac01927058216ade6f72b3d21cc609bafdbf44c1b956191a5]
DEBUG:pyopencl.cache:build program: binary cache hit (key: af95eb07a92eba2c2ba429500126f90f)
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=9830e4e464ac850c53a15d29fb1ea0822392841646effb7ab7351bc85139fdf0]
DEBUG:pytools.persistent_dict:pyopencl-invoker-cache-v7: in mem cache hit [key=b0be9d4383